In [7]:
# import all 5 selected models
import sklearn
import pickle
infile = open('model4web.pickle',"rb")
project_dic = pickle.load(infile)
infile.close()

In [8]:
project_dic["KIRP.pickle"]["genes"]

['ENSG00000187908',
 'ENSG00000167550',
 'ENSG00000274370',
 'ENSG00000258413',
 'ENSG00000124191',
 'ENSG00000095752',
 'ENSG00000225492',
 'ENSG00000101447',
 'ENSG00000256581',
 'ENSG00000149782',
 'ENSG00000230461',
 'ENSG00000219470',
 'ENSG00000108511',
 'ENSG00000154783',
 'ENSG00000275329',
 'ENSG00000144158',
 'ENSG00000223829',
 'ENSG00000243517',
 'ENSG00000257923',
 'ENSG00000168612',
 'ENSG00000234017',
 'ENSG00000177565',
 'ENSG00000197880',
 'ENSG00000174564',
 'ENSG00000198467',
 'ENSG00000212695',
 'ENSG00000226673',
 'ENSG00000254370',
 'ENSG00000230733',
 'ENSG00000267573',
 'ENSG00000151176',
 'ENSG00000273682',
 'ENSG00000088899',
 'ENSG00000250348',
 'ENSG00000129173',
 'ENSG00000101255',
 'ENSG00000237352',
 'ENSG00000136928',
 'ENSG00000232160',
 'ENSG00000130751',
 'ENSG00000137193',
 'ENSG00000229022']

In [9]:
#getting the list of project to run on

In [10]:
#getting the list of the files in the folder
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

'''
    For the given path, get the List of all files in the directory tree 
'''

import pandas as pd
ENSG_key = pd.read_csv("biomart_database.txt")
ENSG_key = ENSG_key.drop(["Transcript stable ID","HGNC symbol","HGNC ID","Protein stable ID","RefSeq match transcript","Transcript name"],axis=1)
ENSG_key = ENSG_key.drop_duplicates()
ENSG_key = ENSG_key.set_index("Gene stable ID")

def ENSG_to_gene(list_of_genes):
    ENSG_key_dic = ENSG_key.to_dict()["Gene name"]
    list_HUGO = []
    for ENSG in list_of_genes:
        if "." in ENSG:
            gene = ENSG.split(".")[0]
        else:
            gene = ENSG
        try:
            print(ENSG_key_dic[gene])
            list_HUGO.append(ENSG_key_dic[gene])
        except KeyError:
            list_HUGO.append(gene)
            print(gene)
    return(list_HUGO)
    
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def get_clinical_data(clinical_file):
    #clinical
    clinical_prp = pd.read_csv(clinical_file,delimiter="\t",header=None)
    clinical_prp = clinical_prp.set_index(0)
    clinical_prp = clinical_prp.transpose()
    clinical_prp["patient.bcr_patient_barcode"] = clinical_prp["patient.bcr_patient_barcode"].str.upper()
    clinical_prp = clinical_prp.set_index("patient.bcr_patient_barcode")
    clinical = clinical_prp
    return(clinical)

def get_dead_alive_stats_from_clinical(clinical_db):
    try: 
        c = clinical_db[["patient.days_to_death","patient.days_to_last_followup","patient.vital_status","patient.person_neoplasm_cancer_status"]]
    except KeyError: return(0,100)
    c["patient.days_to_death"] = pd.to_numeric(c["patient.days_to_death"])
    c["patient.days_to_last_followup"] = pd.to_numeric(c["patient.days_to_last_followup"])
    c = c.replace(np.NaN,-1)
    #dead_ls = list(c[(c["patient.days_to_last_followup"]==-1.0) & (c["patient.days_to_death"]<survive_till)].index)
    #alive_ls = list(c[(c["patient.days_to_last_followup"]>survive_till) & (c["patient.days_to_death"]==-1)].index)
    dead_ls = list(c[(c["patient.vital_status"]=="dead")].index)
    alive_ls = list(c[(c["patient.days_to_last_followup"]>survive_till) & \
                      (c["patient.person_neoplasm_cancer_status"]=="tumor free")\
                      & (c["patient.vital_status"]=="alive")].index)
    #print("dead:",len(dead_ls),"alive:",len(alive_ls))
    return(dead_ls,alive_ls)

def get_X_Y(data_frame,full_alive_ls,full_dead_ls):
    Y =[]
    #prepere Y
    for sample in list(data_frame.index):
        if (sample in full_alive_ls) and (sample in full_dead_ls): print("ERR:",sample)
        if sample in full_alive_ls: Y.append(1)
        if sample in full_dead_ls: Y.append(0)
    X = data_frame.values.tolist()
    return(X,Y)

def get_AUC(y,pred):
    import numpy as np
    from sklearn import metrics
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=None)
    return(metrics.auc(fpr, tpr))

#creare module and calc on data
def RF_moudule(rna,dead_ls,alive_ls,alive_ls_test = [] ,dead_ls_test = [] ):
    #mixing the lists           
    alive_dead_ls = dead_ls+alive_ls
    import random
    random.seed(1)
    random.shuffle(alive_dead_ls)
    list_len = len(alive_dead_ls)

    # splitting the list berween train ,val and test
    alive_dead_ls_train = alive_dead_ls[:int(list_len*0.7)]
    alive_dead_ls_val = alive_dead_ls[int(list_len*0.7):]

    #train the model
    rna = rna.fillna(0)
    X,Y = get_X_Y(rna.loc[alive_dead_ls_train,:].fillna(0),alive_ls,dead_ls)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    clf = RandomForestClassifier(max_depth=2, random_state=0, n_estimators=400)
    parameters = { 'max_features':np.arange(5,10),'n_estimators':[100,500,1000],'min_samples_leaf': [10,50,100],"max_depth":[2,5,10]}
    #random_grid = GridSearchCV(rf, parameters, cv = 5)
    clf.fit(X, Y)

    #val the model
    X,Y = get_X_Y(rna.loc[alive_dead_ls_val,:].fillna(0),alive_ls,dead_ls)
    
    pred = clf.predict_proba(X)
    #print("pred",pred,Y)
    y = np.array(Y)
    pred = pred[:,1]
    auc_val = get_AUC(y,pred)
    print("features:",len(rna.columns),"valAUC-ROC",auc_val)
    
    #test the model on the trainning data
    X,Y = get_X_Y(rna.loc[alive_dead_ls_train,:].fillna(0),alive_ls,dead_ls)
    pred = clf.predict_proba(X)
    y = np.array(Y)
    pred = pred[:,1]
    auc_train_data = get_AUC(y,pred)
    print("features:",len(rna.columns),"trainning_AUC-ROC",auc_train_data)
    
    #test on C3
    if (alive_ls_test != [] and dead_ls_test != []):
        #alive_C3 = ["C3L-00008","C3L-00032","C3L-00098","C3L-00136","C3L-00137","C3L-00143","C3L-00145","C3L-00156","C3L-00157","C3L-00362","C3L-00413","C3L-00449","C3L-00905","C3L-00921","C3L-00942","C3L-00946","C3L-00949","C3L-00961","C3L-01312","C3N-00323","C3N-00880"]
        #dead_C3 = ["C3L-00771","C3L-01307","C3L-00139","C3N-00200","C3N-01349","C3N-01520","C3N-01003"]
        X,Y = get_X_Y(rna.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_ls_test)
        pred = clf.predict_proba(X)
        #print("pred",pred,Y)
        y = np.array(Y)
        pred = pred[:,1]
        auc_c3 = get_AUC(y,pred)
        #print("predictions",clf.predict(X),Y)
        print("features:",len(rna.columns),"test_C3_AUC-ROC",auc_c3)
    else:
        auc_c3=0
    #sorting the genes by importance
    genes_importance = pd.DataFrame(clf.feature_importances_, index = rna.columns)
    genes_importance_sorted = genes_importance.sort_values(by =[0],ascending=False)
    return (clf,auc_val,auc_train_data,auc_c3,list(genes_importance_sorted.index))

def get_FPKM_values(metadata,parent_dir,dead_ls,alive_ls):
    list_of_FPKM_files = []
    c=0
    count_avilable_dead = 0
    count_avilable_alive = 0
    
    for index,line in metadata.iterrows():
        if index not in dead_ls+alive_ls: continue
        c+=1
        #print(index)
        #first DF, creating the base for joinning more DF
        if c==1:
            all_FPKM = pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression='gzip',delimiter="\t",index_col=0,names=[index])
            continue
        try:
            all_FPKM = all_FPKM.join(pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression='gzip',delimiter="\t",index_col=0,names=[index]))
            if index in dead_ls: count_avilable_dead+=1
            if index in alive_ls: count_avilable_alive+=1
        except ValueError: continue
    print("avil alive",count_avilable_alive,"avil dead",count_avilable_dead)
    return all_FPKM

"""def ENSG_to_gene(list_of_genes):
    ENSG_dic = ENSG_key.to_dict()["Gene name"]
    for ENSG in list_of_genes:
        if "." in ENSG:
            gene = ENSG.split(".")[0]
        else:
            gene = ENSG
        try:
            print(ENSG_key_dict[gene])
        except KeyError:
            print(gene)"""
            
"""def get_FPKM_values(metadata,parent_dir,dead_ls,alive_ls):
    list_of_FPKM_files = []
    c=0
    for index,line in metadata.iterrows():
        if index not in dead_ls+alive_ls: continue
        c+=1
        #print(index)
        #first DF, creating the base for joinning more DF
        if c==1:
            all_FPKM = pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression='gzip',delimiter="\t",index_col=0,names=[index])
            continue
        try:
            all_FPKM = all_FPKM.join(pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression='gzip',delimiter="\t",index_col=0,names=[index]))
        except ValueError: continue
    return all_FPKM"""

'def get_FPKM_values(metadata,parent_dir,dead_ls,alive_ls):\n    list_of_FPKM_files = []\n    c=0\n    for index,line in metadata.iterrows():\n        if index not in dead_ls+alive_ls: continue\n        c+=1\n        #print(index)\n        #first DF, creating the base for joinning more DF\n        if c==1:\n            all_FPKM = pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression=\'gzip\',delimiter="\t",index_col=0,names=[index])\n            continue\n        try:\n            all_FPKM = all_FPKM.join(pd.read_csv(parent_dir+"\\"+line["file_id"]+"\\"+line["file_name"], compression=\'gzip\',delimiter="\t",index_col=0,names=[index]))\n        except ValueError: continue\n    return all_FPKM'

In [11]:
from IPython.display import clear_output
def RF_moudule_GridSearch(rna,dead_ls,alive_ls):
    #mixing the lists           
    alive_dead_ls = dead_ls+alive_ls
    import random
    random.seed(1)
    random.shuffle(alive_dead_ls)
    list_len = len(alive_dead_ls)

    # splitting the list berween train ,val and test
    #alive_dead_ls_train = alive_dead_ls[:int(list_len*0.7)]
    #alive_dead_ls_val = alive_dead_ls[int(list_len*0.7):]

    #train the model
    rna = rna.fillna(0)
    X,Y = get_X_Y(rna.loc[alive_dead_ls,:].fillna(0),alive_ls,dead_ls)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    rf = RandomForestClassifier()
    #parameters = { 'max_features':np.arange(1,5,10),'n_estimators':[100,300,500,1000],'min_samples_leaf': [10,50,100],"max_depth":[2,5,10],random_state=[0]}
    parameters = {'bootstrap': [True, False], 'max_depth': [10, 50, 100, None], 'max_features': ['auto', 'sqrt'], 'min_samples_leaf': [1, 2, 4],\
    'min_samples_split': [2, 5, 10], 'n_estimators': [200, 600, 1000],"random_state":[0]}
    
    #parameters = { 'max_depth':[2],'n_estimators':[400]} #this is just for testing
    #max_depth=2, random_state=0, n_estimators=400
    
    random_grid = GridSearchCV(rf, parameters, cv = 3, refit=True, verbose=2,n_jobs=1)
    clear_output(wait=True)
    random_grid.fit(X, Y)
    best_clf = random_grid.best_estimator_
    return (best_clf)

In [12]:
#dirName = "TCGA_uterus_CESC_all_patiets.tar/";
# Get the list of all files in directory tree at given path
#listOfFiles = getListOfFiles(dirName)


payload_start = """{
    "filters":{
        "op":"in",
        "content":{
            "field":"files.file_id",
            "value":["""
payload_end = """            ]
        }
    },
    "format":"TSV",
    "fields":"file_id,file_name,cases.submitter_id,cases.case_id,data_category,data_type,cases.samples.tumor_descriptor,cases.samples.tissue_type,cases.samples.sample_type,cases.samples.submitter_id,cases.samples.sample_id,cases.samples.portions.analytes.aliquots.aliquot_id,cases.samples.portions.analytes.aliquots.submitter_id",
    "size":"10000"
}"""
#geti all the files list from the lib
list_of_dir = ["all_TCGA_projects"]
listOfFiles = []
for loc in list_of_dir:
    listOfFiles+=getListOfFiles(loc)
    
#creating a list of the files for the payload
string_of_files_labels = ""
for file_name in listOfFiles:
    if file_name[-3:]!=".gz": continue
    file_label = file_name.split("\\")[1]
    string_of_files_labels+="\""+file_label+"\",\n"
    
string_of_files_labels=string_of_files_labels[:-2]+"\n"
#print(string_of_files_labels)
#creating the Payload_txt file (working accoring the instruction of https://docs.gdc.cancer.gov/API/Users_Guide/Search_and_Retrieval/#example-http-post-request)
file = open("Payload.txt","w")
file.write(payload_start+string_of_files_labels+payload_end)
file.close()

print("Please run from this path in cygwin the following command: curl --request POST --header \"Content-Type: application/json\" --data @Payload.txt 'https://api.gdc.cancer.gov/files' > File_metadata.txt")


Please run from this path in cygwin the following command: curl --request POST --header "Content-Type: application/json" --data @Payload.txt 'https://api.gdc.cancer.gov/files' > File_metadata.txt


In [13]:
#calculating which project have the 0.2<alive/(alive+dead)<0.8 ratio using the clinical data files downloaded from firehose
survive_till = 1095 # 1095 days  = 3 years of survival without a tumer
dead_alive_stats = []
projects_for_analysis= {}
projects_not_for_analysis = []
import glob, re
list_of_clinical_files = glob.glob("clinical/*")
for clinic_file in list_of_clinical_files: 
    project_name = re.findall("[_][\D]{2,9}[.]",clinic_file)
    TCGA_project_name = project_name[0].replace("_","").replace(".","")
    clinical_location = clinic_file+"\\"+TCGA_project_name+".clin.merged.txt"
    clinical_df = get_clinical_data(clinical_location)
    #print("----------------")
    #print("TCGA_project_name:",TCGA_project_name)
    dead_ls,alive_ls=get_dead_alive_stats_from_clinical(clinical_df)
    try:
        alive_per = len(alive_ls)/(len(dead_ls)+len(alive_ls))
        print(TCGA_project_name,"alive:",len(alive_ls)," | dead",len(dead_ls), " | alive %:",alive_per)
    except TypeError:
        print(alive_ls,dead_ls)
        

    if (alive_per>0.2 and alive_per<0.8): 
        projects_for_analysis[TCGA_project_name]={"alive_ls":alive_ls,"dead_ls":dead_ls}
        #if the project is for analysis, first we will calculte the cls, for the whole 65K~ features
        
        #getting the RNA values from the files
        #FPKM_project_df = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_ls,alive_ls)
        #FPKM_project_df_t = FPKM_project_df.transpose()
        
        #best_clf = RF_moudule_GridSearch(FPKM_project_df_t,dead_ls,alive_ls)
        #getting the best features out of the 65K~
       
        #genes_importance = pd.DataFrame(best_clf.feature_importances_, index = FPKM_project_df_t.columns)
        #genes_importance_sorted = genes_importance.sort_values(by =[0],ascending=False)
        #genes_importance_sorted_fltr = genes_importance_sorted[genes_importance_sorted[0]>0]
        #print(genes_importance_sorted_fltr.head(10))
        
        
    else: 
        projects_not_for_analysis.append(TCGA_project_name)
#print(projects_for_analysis)

BLCA alive: 49  | dead 109  | alive %: 0.310126582278481
BRCA alive: 193  | dead 104  | alive %: 0.6498316498316499
CESC alive: 53  | dead 60  | alive %: 0.4690265486725664
COAD alive: 20  | dead 57  | alive %: 0.2597402597402597
COADREAD alive: 23  | dead 66  | alive %: 0.25842696629213485
ESCA alive: 7  | dead 57  | alive %: 0.109375
GBM alive: 3  | dead 446  | alive %: 0.0066815144766146995
GBMLGG alive: 31  | dead 538  | alive %: 0.054481546572934976
HNSC alive: 50  | dead 170  | alive %: 0.22727272727272727
KICH alive: 48  | dead 10  | alive %: 0.8275862068965517
KIPAN alive: 246  | dead 204  | alive %: 0.5466666666666666
KIRC alive: 159  | dead 162  | alive %: 0.4953271028037383
KIRP alive: 39  | dead 32  | alive %: 0.5492957746478874
100 0
LGG alive: 28  | dead 92  | alive %: 0.23333333333333334
LIHC alive: 32  | dead 91  | alive %: 0.2601626016260163
LUAD alive: 39  | dead 127  | alive %: 0.23493975903614459
LUSC alive: 49  | dead 161  | alive %: 0.23333333333333334
OV alive: 4

IndexError: list index out of range

In [15]:
#import file-sample keys, and cleaning all the normal tissue RNA-seq samples
import pandas as pd
metadata = pd.read_csv("File_metadata.txt",delimiter="\t")
metadata.head()
metadata = metadata[["cases.0.submitter_id","file_id","file_name","cases.0.samples.0.sample_type"]]
metadata = metadata.set_index("cases.0.submitter_id")
metadata = metadata[metadata["cases.0.samples.0.sample_type"]!="Solid Tissue Normal"]
metadata.groupby(["cases.0.samples.0.sample_type"]).count()

#get the ENSG key
import pandas as pd
ENSG_key = pd.read_csv("biomart_database.txt")
ENSG_key = ENSG_key.drop(["Transcript stable ID","HGNC symbol","HGNC ID","Protein stable ID","RefSeq match transcript","Transcript name"],axis=1)
ENSG_key = ENSG_key.drop_duplicates()
ENSG_key = ENSG_key.set_index("Gene stable ID")

In [ ]:
projects_auc_0_7 = []
projects_res ={}

for project in list(project_dic.keys()):
    print("checking prediction model for project: ",project)
    clf_ = project_dic[project]['clf']
    genes_ = project_dic[project]['genes']
    

    for TCGA_project in list(projects_for_analysis.keys()):
    #for TCGA_project in ["KIRC"]:
        #getting the samples and prepering the DataFrame
        print("---------------------------------------------------------")
        print(TCGA_project)
        #if TCGA_project=="BLCA": continue
        #if TCGA_project!="BRCA": continue
        alive_ls = projects_for_analysis[TCGA_project]["alive_ls"]
        dead_ls = projects_for_analysis[TCGA_project]["dead_ls"]
        try:
            if len(alive_ls)==0 or len(dead_ls)==0:
                print("ERROR:",alive_ls,dead_ls)
                continue
        except TypeError:
            print("ERROR:",alive_ls,dead_ls)
            continue

        #extracting the clinical values from the files
        FPKM_project_df = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_ls,alive_ls)
        FPKM_project_df_t = FPKM_project_df.transpose()
        
        avl_samples = FPKM_project_df_t.index.tolist()
        #filtering out not avilable samples
        alive_ls_new = []
        dead_ls_new =[]
        for sam in alive_ls:
            if sam in avl_samples: alive_ls_new.append(sam)
        for sam in dead_ls:
            if sam in avl_samples: dead_ls_new.append(sam)
        
        #changing the names of the genes 
        x = FPKM_project_df_t.loc[alive_ls_new+dead_ls_new,:].fillna(0)
        b = [el[0] for el in x.columns.str.split(".")]
        FPKM_project_df_t.columns = b
        
        #working with filtering the genes for the model
        FPKM_project_df_t = FPKM_project_df_t[genes_]
        
  
        X,Y = get_X_Y(FPKM_project_df_t.loc[alive_ls_new+dead_ls_new,:].fillna(0),alive_ls_new,dead_ls_new)
        # testing the classfier
        pred = clf_.predict_proba(X)
        
        #print("pred",pred,Y)
        y = np.array(Y)
        pred = pred[:,1]
        auc_all = get_AUC(y,pred)
        print(project,TCGA_project,auc_all)

In [ ]:
FPKM_project_df_t.loc[alive_ls_new+dead_ls_new,:]

In [ ]:
FPKM_project_df_t.index.tolist()

## normalization test for the KIRP model

In [17]:
# extracting the model from the dictinary
KIRP_clf,KIRP_genes = project_dic["KIRP.pickle"]["clf"],project_dic["KIRP.pickle"]["genes"]

In [10]:
#getting the KIRP-TCGA samples
alive_ls = projects_for_analysis["KIRP"]["alive_ls"]
dead_ls = projects_for_analysis["KIRP"]["dead_ls"]
try:
    if len(alive_ls)==0 or len(dead_ls)==0:
        print("ERROR:",alive_ls,dead_ls)
except TypeError:
    print("ERROR:",alive_ls,dead_ls)

#extracting the clinical values from the files
FPKM_project_df = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_ls,alive_ls)
FPKM_project_df_t = FPKM_project_df.transpose()

avl_samples = FPKM_project_df_t.index.tolist()
#filtering out not avilable samples
alive_ls_new = []
dead_ls_new =[]
for sam in alive_ls:
    if sam in avl_samples: alive_ls_new.append(sam)
for sam in dead_ls:
    if sam in avl_samples: dead_ls_new.append(sam)

#changing the names of the genes 
x = FPKM_project_df_t.loc[alive_ls_new+dead_ls_new,:].fillna(0)
b = [el[0] for el in x.columns.str.split(".")]
FPKM_project_df_t.columns = b

#working with filtering the genes for the model
FPKM_project_df_t = FPKM_project_df_t[KIRP_genes]


X,Y = get_X_Y(FPKM_project_df_t.loc[alive_ls_new+dead_ls_new,:].fillna(0),alive_ls_new,dead_ls_new)

avil alive 37 avil dead 32


In [18]:
#extracting the CPTAC3 samples
alive_C3_renal = ["C3L-00010","C3L-00026","C3L-00097","C3L-00359","C3L-00360","C3L-00418","C3L-00447","C3N-00312","C3N-00313"]
dead_C3_renal = ["C3L-00011","C3L-00079","C3L-00096","C3L-00369","C3L-01302","C3N-00314","C3N-00491","C3N-00831","C3N-01200","C3L-00088","C3L-01287","C3N-01220"]
FPKM_project_df_C3 = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_C3_renal,alive_C3_renal)
FPKM_project_df_t_C3 = FPKM_project_df_C3.transpose()

#changing the names of the genes 
x = FPKM_project_df_t_C3.loc[alive_C3_renal+dead_C3_renal,:].fillna(0)
b = [el[0] for el in x.columns.str.split(".")]
FPKM_project_df_t_C3.columns = b
        
#working with filtering the genes for the model
FPKM_project_df_t_C3 = FPKM_project_df_t_C3[KIRP_genes]
#X,Y = get_X_Y(FPKM_project_df_t.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_C3)
X_C3,Y_C3 =get_X_Y(FPKM_project_df_t_C3.loc[alive_C3_renal+dead_C3_renal,:].fillna(0),alive_C3_renal,dead_C3_renal)
print(Y_C3,KIRP_clf.predict_proba(X_C3)[:,1])

avil alive 8 avil dead 12
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.355 0.325 0.255 0.38  0.3   0.295 0.195 0.325 0.435 0.39  0.21  0.29
 0.3   0.16  0.17  0.145 0.215 0.195 0.3   0.185 0.17 ]


In [35]:
FPKM_val_to_ext

,C3L-00010,C3L-00026,C3L-00097,C3L-00359,C3L-00360,C3L-00418,C3L-00447,C3N-00312,C3N-00313,C3L-00011,...,C3L-00096,C3L-00369,C3L-01302,C3N-00314,C3N-00491,C3N-00831,C3N-01200,C3L-00088,C3L-01287,C3N-01220
ENSG00000187908,619.067100,5910.407100,2021.508700,521.282200,1922.855900,1173.749400,743.181000,1173.855400,1879.289500,313.475100,...,123.637900,5397.991000,9585.522600,9528.437600,14451.125500,2317.264800,826.645800,1210.453000,808.567700,972.616700
ENSG00000167550,14792.430500,14405.911900,21811.868300,7747.860200,13752.571900,5386.691000,23254.651200,15042.117900,4602.074000,17263.584500,...,11993.017000,32003.413700,22833.966100,19166.879600,21339.105100,18220.168400,17111.567400,14729.523900,20601.956600,53911.402700
ENSG00000274370,20820.769600,15618.565000,4605.124400,17218.950100,19054.683500,15508.476400,13747.266800,16248.441600,56138.775100,4141.873200,...,2450.397300,4504.572100,39221.037500,34471.802100,7508.850400,22766.842600,27725.780300,8885.240000,3815.505800,36717.021000
ENSG00000258413,15185.124900,10743.533100,8275.679400,19691.296700,5188.240500,11823.485500,31281.858400,18098.805000,24692.054800,18269.647800,...,6004.783800,16557.915900,19355.992700,14691.191200,9638.461300,22444.630700,14412.149000,14515.726300,8726.694300,14996.056000
ENSG00000124191,13955.803500,16362.804200,30206.358300,2284.996900,11008.884700,20580.128300,30901.340400,20581.985700,4297.934500,8637.148600,...,12077.881400,19640.940800,8674.546400,26139.957900,16065.099000,12352.754300,18157.480800,22065.885700,12730.506700,3132.282000
ENSG00000095752,170.100700,376.084000,0.000000,0.000000,0.000000,473.015300,898.494900,0.000000,987.840700,568.480200,...,560.536400,343.478500,186.915400,0.000000,2208.438200,359.171700,1921.928200,0.000000,1221.933200,3359.655500
ENSG00000225492,114550.590800,87340.310300,39956.894400,15432.264400,50914.336600,41093.506300,183664.286200,101112.195800,7572.290100,9253.353400,...,171871.618300,32960.431900,29292.795500,37377.588400,92237.452100,106662.163600,124189.819700,114486.619800,67004.304900,16215.123200
ENSG00000101447,22946.411000,31860.544200,21600.359200,7864.016300,26432.026400,25523.714700,37089.054600,18050.541500,9061.592200,67893.920900,...,136915.001500,10935.038500,33081.690600,28490.028500,112281.259300,32753.485200,100180.449300,14038.320200,64239.551400,48493.912800
ENSG00000256581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,172.774200,188.041900,344.915600,493.721600,180.668100,193.351000,0.000000,351.227700,633.731600
ENSG00000149782,102326.940200,98549.840100,119713.938100,79266.139700,95490.023000,121404.752200,138993.856100,122088.422900,101256.753100,128675.419100,...,86662.648500,121701.942100,136881.704000,146844.828600,116717.805500,126875.851700,136506.460500,115489.504900,114966.939000,150268.698500


In [37]:
a = [12,34,53]
a/max(a)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [34]:
 pd.set_option('float_format', '{:f}'.format)
FPKM_val_to_ext = FPKM_project_df_t_C3.loc[alive_C3_renal+dead_C3_renal,:].transpose()
#extract values for the websites site demo
for i in alive_C3_renal+dead_C3_renal:
    if i in alive_C3_renal:
        dead_alive = "survived"
    else:
        dead_alive = "not survived"
    print("@" + i + " "  + dead_alive)
    print(FPKM_val_to_ext[i])


@C3L-00010 survived
ENSG00000187908       619.067100
ENSG00000167550     14792.430500
ENSG00000274370     20820.769600
ENSG00000258413     15185.124900
ENSG00000124191     13955.803500
ENSG00000095752       170.100700
ENSG00000225492    114550.590800
ENSG00000101447     22946.411000
ENSG00000256581         0.000000
ENSG00000149782    102326.940200
ENSG00000230461        44.153400
ENSG00000219470     13908.334800
ENSG00000108511     11515.503000
ENSG00000154783    121574.910600
ENSG00000275329      1232.607000
ENSG00000144158      1774.761500
ENSG00000223829       295.601100
ENSG00000243517     27740.166100
ENSG00000257923    131412.965500
ENSG00000168612    103862.963900
ENSG00000234017     64097.609300
ENSG00000177565   1032191.642700
ENSG00000197880      2545.316200
ENSG00000174564     22046.369800
ENSG00000198467    278530.986500
ENSG00000212695     17112.577700
ENSG00000226673      1729.716800
ENSG00000254370      6223.821100
ENSG00000230733     15372.895400
ENSG00000267573       6

In [36]:
def calc_mean_for_each_group(Y_values,pred_for_X):
        #print(Y_values,pred_for_X)
        res = pd.DataFrame([Y_values,pred_for_X])
        res_t = res.transpose()
        res_t.columns = ["true_values","pred_prob"]
        #print(("1.0 mean:",res_t[res_t["true_values"]==1.0]["pred_prob"].mean(),"0.0 mean:",res_t[res_t["true_values"]==0.0]["pred_prob"].mean()))
        return({"1.0 mean":res_t[res_t["true_values"]==1.0]["pred_prob"].mean(),"0.0 mean":res_t[res_t["true_values"]==0.0]["pred_prob"].mean()})
calc_mean_for_each_group(Y_C3,KIRP_clf.predict_proba(X_C3)[:,1])

def sep_survive_not_survive_scores(Y_values,pred_for_X):
        #print(Y_values,pred_for_X)
        res = pd.DataFrame([Y_values,pred_for_X])
        res_t = res.transpose()
        res_t.columns = ["true_values","pred_prob"]
        #print(("1.0 mean:",res_t[res_t["true_values"]==1.0]["pred_prob"].mean(),"0.0 mean:",res_t[res_t["true_values"]==0.0]["pred_prob"].mean()))
        return(list(res_t[res_t["true_values"]==1.0]["pred_prob"].values),list(res_t[res_t["true_values"]==0.0]["pred_prob"].values))

    #calc_mean_for_each_group(Y_C3,KIRP_clf.predict_proba(X_C3)[:,1])

def create_boxplot(train_Y,train_pred,test_Y,test_pred,C3_Y,C3_pred,iter_num,norm_or_not):
    train_surv_values,train_not_surv_values = sep_survive_not_survive_scores(train_Y,train_pred)
    test_surv_values,test_not_surv_values = sep_survive_not_survive_scores(test_Y,test_pred)
    C3_surv_values,C3_not_surv_values = sep_survive_not_survive_scores(C3_Y,C3_pred)
    import matplotlib.pyplot as plt
    import numpy as np
    data = [train_surv_values,train_not_surv_values,test_surv_values,test_not_surv_values,C3_surv_values,C3_not_surv_values]
    #print(data)

    fig = plt.figure(figsize =(10, 7))

    # Creating axes instance
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xticklabels(["TRAIN_surv","TRAIN_not_surv","TEST_surv","TEST_not_surv","C3_surv","C3_not_surv"])
    # Creating plot
    bp = ax.boxplot(data)
    ax.set_title(str(i)+"_"+norm_or_not)
    #fig.title("Test"+str(i)+" "+norm_or_not)
    #bp.set_xticklabels(["TRAIN_surv","TRAIN_not_surv","TEST_surv","TEST_not_surv","C3_surv","C3_not_surv"])
    
    # show plot
    #plt.show()
    #plt.title("Test"+str(i)+" "+norm_or_not)
    plt.savefig("test_KIRP_norm\\"+str(i)+"_"+norm_or_not+".png",format="png",bbox_inches='tight')
    plt.close()

In [37]:
#with normalization
import qnorm
CPTAC=[]
CPTAC_AUC = []
KIRP_test =[]
KIRP_test_AUC = []
KIRP_train = []
KIRP_train_AUC = []
for i in range(1,10):
    print("test ",i)
    import random
    FPKM_project_df_C3 = FPKM_project_df_t_C3.transpose()
    FPKM_project_df = FPKM_project_df_t.transpose()
    #create one table from both groups
    FPKM_C3_KIRP = FPKM_project_df.join(FPKM_project_df_C3)
    #normalize the table 
    FPKM_C3_KIRP_norm_t = qnorm.quantile_normalize(FPKM_C3_KIRP, axis=1).transpose()
    #extracting the traning and testign set 
    X,Y = get_X_Y(FPKM_C3_KIRP_norm_t.loc[alive_ls_new+dead_ls_new,:].fillna(0),alive_ls_new,dead_ls_new)
    c = list(zip(X, Y))
    random.shuffle(c)
    X, Y = zip(*c)
    X_train = X[:int(len(X)*0.7)]
    Y_train = Y[:int(len(X)*0.7)]
    
    #tranning the model 
    KIRP_clf.fit(X_train,Y_train)
    
    X_C3,Y_C3 =get_X_Y(FPKM_C3_KIRP_norm_t.loc[alive_C3_renal+dead_C3_renal,:].fillna(0),alive_C3_renal,dead_C3_renal)
    #predictions with CPTAC3 data
    C3_pred = KIRP_clf.predict_proba(X_C3) 
    Y_C3 = np.array(Y_C3)
    C3_pred = C3_pred[:,1]
    C3_auc = get_AUC(Y_C3,C3_pred)
    CPTAC_AUC.append(auc_all)
    CPTAC.append(calc_mean_for_each_group(Y_C3,C3_pred))
    
    X_test = X[int(len(X)*0.7):]
    Y_test = Y[int(len(X)*0.7):]
    test_pred = KIRP_clf.predict_proba(X_test) 
    Y_test = np.array(Y_test)
    test_pred = test_pred[:,1]
    test_auc = get_AUC(Y_test,test_pred)
    train_pred = KIRP_clf.predict_proba(X_train) 
    Y_train = np.array(Y_train)
    train_pred = train_pred[:,1]
    train_auc = get_AUC(Y_train,train_pred)

    
    create_boxplot(list(Y_train),list(train_pred),list(Y_test),list(test_pred),list(Y_C3),list(C3_pred),i,"normalize")

test  1
test  2
test  3
test  4
test  5
test  6
test  7
test  8
test  9


In [38]:
#not normalize
import qnorm
CPTAC=[]
CPTAC_AUC = []
KIRP_test =[]
KIRP_test_AUC = []
KIRP_train = []
KIRP_train_AUC = []
for i in range(1,10):
    print("test ",i)
    import random
    FPKM_project_df_C3 = FPKM_project_df_t_C3.transpose()
    FPKM_project_df = FPKM_project_df_t.transpose()
    #create one table from both groups
    FPKM_C3_KIRP = FPKM_project_df.join(FPKM_project_df_C3)
    #normalize the table 
    FPKM_C3_KIRP_t = FPKM_C3_KIRP.transpose()
    #extracting the traning and testign set 
    X,Y = get_X_Y(FPKM_C3_KIRP_t.loc[alive_ls_new+dead_ls_new,:].fillna(0),alive_ls_new,dead_ls_new)
    c = list(zip(X, Y))
    random.shuffle(c)
    X, Y = zip(*c)
    X_train = X[:int(len(X)*0.7)]
    Y_train = Y[:int(len(X)*0.7)]
    
    #tranning the model 
    KIRP_clf.fit(X_train,Y_train)
    
    X_C3,Y_C3 =get_X_Y(FPKM_C3_KIRP_t.loc[alive_C3_renal+dead_C3_renal,:].fillna(0),alive_C3_renal,dead_C3_renal)
    #predictions with CPTAC3 data
    C3_pred = KIRP_clf.predict_proba(X_C3) 
    Y_C3 = np.array(Y_C3)
    C3_pred = C3_pred[:,1]
    C3_auc = get_AUC(Y_C3,C3_pred)
    CPTAC_AUC.append(auc_all)
    CPTAC.append(calc_mean_for_each_group(Y_C3,C3_pred))
    
    X_test = X[int(len(X)*0.7):]
    Y_test = Y[int(len(X)*0.7):]
    test_pred = KIRP_clf.predict_proba(X_test) 
    Y_test = np.array(Y_test)
    test_pred = test_pred[:,1]
    test_auc = get_AUC(Y_test,test_pred)
    train_pred = KIRP_clf.predict_proba(X_train) 
    Y_train = np.array(Y_train)
    train_pred = train_pred[:,1]
    train_auc = get_AUC(Y_train,train_pred)

    
    create_boxplot(list(Y_train),list(train_pred),list(Y_test),list(test_pred),list(Y_C3),list(C3_pred),i,"not normalize")

test  1
test  2
test  3
test  4
test  5
test  6
test  7
test  8
test  9


In [98]:
KIRP_test

[{'1.0 mean': 0.742, '0.0 mean': 0.43227272727272725},
 {'1.0 mean': 0.8378571428571427, '0.0 mean': 0.41321428571428565},
 {'1.0 mean': 0.6741666666666667, '0.0 mean': 0.30166666666666664},
 {'1.0 mean': 0.71375, '0.0 mean': 0.4292307692307693},
 {'1.0 mean': 0.7050000000000001, '0.0 mean': 0.39888888888888885},
 {'1.0 mean': 0.6616666666666667, '0.0 mean': 0.4605555555555555},
 {'1.0 mean': 0.6473333333333334, '0.0 mean': 0.2991666666666667},
 {'1.0 mean': 0.6657692307692307, '0.0 mean': 0.33625},
 {'1.0 mean': 0.7194444444444444, '0.0 mean': 0.4358333333333333},
 {'1.0 mean': 0.6477272727272727, '0.0 mean': 0.31399999999999995},
 {'1.0 mean': 0.595, '0.0 mean': 0.2675},
 {'1.0 mean': 0.6765384615384616, '0.0 mean': 0.37124999999999997},
 {'1.0 mean': 0.6079166666666667, '0.0 mean': 0.34611111111111115},
 {'1.0 mean': 0.7483333333333333, '0.0 mean': 0.44416666666666665},
 {'1.0 mean': 0.6579166666666666, '0.0 mean': 0.3972222222222222},
 {'1.0 mean': 0.698, '0.0 mean': 0.4},
 {'1.0 m

In [83]:
#without normalization
import random
import random
FPKM_C3_KIRP_t = FPKM_project_df_t_C3.append(FPKM_project_df_t)
#FPKM_C3_KIRP = FPKM_C3_KIRP_t.transpose()
#FPKM_C3_KIRP_norm_t = qnorm.quantile_normalize(FPKM_C3_KIRP, axis=1).transpose()
X,Y = get_X_Y(FPKM_C3_KIRP_t.loc[alive_ls_new+dead_ls_new,:].fillna(0),alive_ls_new,dead_ls_new)
c = list(zip(X, Y))
random.shuffle(c)
X, Y = zip(*c)
X_train = X[:int(len(X)*0.7)]
Y_train = Y[:int(len(X)*0.7)]
KIRP_clf.fit(X_train,Y_train)
X_C3,Y_C3 =get_X_Y(FPKM_C3_KIRP_norm_t.loc[alive_C3_renal+dead_C3_renal,:].fillna(0),alive_C3_renal,dead_C3_renal)
print(Y_C3,KIRP_clf.predict_proba(X_C3)[:,1])
#calc_mean_for_each_group(Y_C3,KIRP_clf.predict_proba(X_C3)[:,1])

#predictions with CPTAC3 data
print("pridiction for CPTAC3 data")
pred = KIRP_clf.predict_proba(X_C3) 
#print("pred",pred,Y)
y_c3 = np.array(Y_C3)
pred = pred[:,1]
auc_all = get_AUC(y_c3,pred)
print(auc_all)
calc_mean_for_each_group(Y_C3,pred)

#predictions using TCGA-KIRP data
print("predictions using TCGA-KIRP test data")
X_test = X[int(len(X)*0.7):]
Y_test = Y[int(len(X)*0.7):]
#print(X_test,Y_test)
pred = KIRP_clf.predict_proba(X_test) 
#print("pred",pred,Y)
y = np.array(Y_test)
print(KIRP_clf.predict_proba(X_test)[:,1],Y_test)
pred = pred[:,1]
auc_all = get_AUC(y,pred)
print(auc_all)
calc_mean_for_each_group(Y_test,pred)

#predictions using TCGA-KIRP data
print("predictions using TCGA-KIRP train data")
pred = KIRP_clf.predict_proba(X_train) 
#print("pred",pred,Y)
y = np.array(Y_train)
print(KIRP_clf.predict_proba(X_train)[:,1],Y_train)
pred = pred[:,1]
auc_all = get_AUC(y,pred)
print(auc_all)
calc_mean_for_each_group(Y_train,pred)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.37  0.415 0.32  0.415 0.335 0.415 0.31  0.385 0.45  0.26  0.215 0.225
 0.285 0.295 0.41  0.245 0.415 0.29  0.36  0.295 0.135]
pridiction for CPTAC3 data
0.8657407407407408
('1.0 mean:', 0.3794444444444445, '0.0 mean:', 0.28583333333333333)
predictions using TCGA-KIRP test data
[0.35  0.615 0.59  0.08  0.43  0.405 0.585 0.675 0.25  0.135 0.36  0.15
 0.665 0.225 0.78  0.695 0.795 0.87  0.795 0.895 0.275] (0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0)
0.9903846153846154
('1.0 mean:', 0.6730769230769231, '0.0 mean:', 0.23375)
predictions using TCGA-KIRP train data
[0.31  0.165 0.795 0.92  0.11  0.175 0.05  0.185 0.03  0.835 0.155 0.175
 0.945 0.93  0.935 0.095 0.88  0.92  0.135 0.1   0.83  0.86  0.095 0.905
 0.94  0.91  0.915 0.79  0.89  0.085 0.935 0.1   0.9   0.9   0.155 0.31
 0.04  0.04  0.18  0.895 0.11  0.865 0.94  0.82  0.935 0.05  0.235 0.12
 0.92 ] (0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,

In [45]:
print(Y,KIRP_clf.predict_proba(X)[:,1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.815 0.91  0.97  0.88  0.915 0.86  0.87  0.965 0.79  0.845 0.935 0.885
 0.77  0.975 0.865 0.9   0.825 0.9   0.855 0.84  0.91  0.925 0.95  0.965
 0.91  0.88  0.96  0.83  0.905 0.825 0.885 0.95  0.74  0.85  0.92  0.9
 0.905 0.955 0.24  0.115 0.145 0.07  0.125 0.155 0.25  0.03  0.105 0.085
 0.09  0.24  0.07  0.04  0.155 0.135 0.105 0.175 0.14  0.21  0.1   0.065
 0.145 0.045 0.225 0.115 0.04  0.095 0.04  0.09  0.26  0.2  ]


NameError: name 'FPKM_C3_KIRP_norm_t' is not defined

In [1]:
import pandas as pd
import qnorm

df = pd.DataFrame({'C1': {'A': 5, 'B': 2, 'C': 3, 'D': 4},
                   'C2': {'A': 4, 'B': 1, 'C': 4, 'D': 2},
                   'C3': {'A': 3, 'B': 4, 'C': 6, 'D': 8}})

print(qnorm.quantile_normalize(df, axis=1))

         C1        C2        C3
A  5.666667  5.166667  2.000000
B  2.000000  2.000000  3.000000
C  3.000000  5.166667  4.666667
D  4.666667  3.000000  5.666667


In [2]:
#combinning the sample
FPKM_project_df_t_C3

NameError: name 'FPKM_project_df_t_C3' is not defined

In [7]:


#get the ENSG key
import pandas as pd
ENSG_key = pd.read_csv("biomart_database.txt")
ENSG_key = ENSG_key.drop(["Transcript stable ID","HGNC symbol","HGNC ID","Protein stable ID","RefSeq match transcript","Transcript name"],axis=1)
ENSG_key = ENSG_key.drop_duplicates()
ENSG_key = ENSG_key.set_index("Gene stable ID")

projects_auc_0_7 = []
projects_res ={}


for TCGA_project in list(projects_for_analysis.keys()):
#for TCGA_project in ["KIRC"]:
    #getting the samples and prepering the DataFrame
    print("---------------------------------------------------------")
    print(TCGA_project)
    #if TCGA_project=="BLCA": continue
    #if TCGA_project!="BRCA": continue
    alive_ls = projects_for_analysis[TCGA_project]["alive_ls"]
    dead_ls = projects_for_analysis[TCGA_project]["dead_ls"]
    try:
        if len(alive_ls)==0 or len(dead_ls)==0:
            print("ERROR:",alive_ls,dead_ls)
            continue
    except TypeError:
        print("ERROR:",alive_ls,dead_ls)
        continue
     
    FPKM_project_df = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_ls,alive_ls)
    FPKM_project_df_t = FPKM_project_df.transpose()
    #testing the best classifier for the 65k features

    clf_best_clf_65k = RF_moudule_GridSearch(FPKM_project_df_t,dead_ls,alive_ls)

    #after we got the best classifier we'll calculate the AUC with it
    print(clf_best_clf_65k)
    
    X,Y = get_X_Y(FPKM_project_df_t.loc[alive_ls+dead_ls,:].fillna(0),alive_ls,dead_ls)
"""
    pred = clf_best_clf_65k.predict_proba(X)
    #print("pred",pred,Y)
    y = np.array(Y)
    pred = pred[:,1]
    auc_all = get_AUC(y,pred)
    print("allAUC-ROC, 65k features: ",auc_all)
    
    #getting the 100-1K features form clf_best_clf_65k
    genes_importance = pd.DataFrame(clf_best_clf_65k.feature_importances_, index = FPKM_project_df_t.columns)
    genes_importance_fltr = genes_importance[genes_importance[0]>0]
    
    # splitting the data to train and val.
    alive_dead_ls = dead_ls+alive_ls
    import random
    random.seed(1)
    random.shuffle(alive_dead_ls)
    list_len = len(alive_dead_ls)
    alive_dead_ls_train = alive_dead_ls[:int(list_len*0.7)]
    alive_dead_ls_val = alive_dead_ls[int(list_len*0.7):]
    
    #finding the best model using the train data for 1k features
    FPKM_project_df_t_train = FPKM_project_df_t[list(genes_importance_fltr.index)].loc[alive_dead_ls_train]
    clf_best_clf_1k = RF_moudule_GridSearch(FPKM_project_df_t_train,dead_ls,alive_ls)
    
    print("clf_best_clf_1k: ",clf_best_clf_1k)
    #getting the auc for the validation data
    FPKM_project_df_t_val = FPKM_project_df_t[list(genes_importance_fltr.index)].loc[alive_dead_ls_val]
    X,Y = get_X_Y(FPKM_project_df_t_val.fillna(0),alive_ls,dead_ls)
    pred = clf_best_clf_1k.predict_proba(X)
    #print("pred",pred,Y)
    y = np.array(Y)
    pred = pred[:,1]
    auc_all = get_AUC(y,pred)
    print("valAUC-ROC, 1k features",auc_all)
    print(FPKM_project_df_t_train.shape)
    project_models_and_scores={}
    rna_top_fet = FPKM_project_df_t[list(FPKM_project_df_t_train.columns)]

    #checing the model on diffrent data set if possible
    alive_C3 = ["C3L-00008","C3L-00032","C3L-00098","C3L-00136","C3L-00137","C3L-00143","C3L-00145","C3L-00156","C3L-00157","C3L-00362","C3L-00413","C3L-00449","C3L-00905","C3L-00921","C3L-00942","C3L-00946","C3L-00949","C3L-00961","C3L-01312","C3N-00323","C3N-00880"]
    dead_C3 = ["C3L-00771","C3L-01307","C3L-00139","C3N-00200","C3N-01349","C3N-01520","C3N-01003"]
    FPKM_project_df_CPTAC = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_C3,alive_C3)
    #genes = projects_res2[model][max_num]['genes']
    FPKM_project_df_CPTAC_uter_t = FPKM_project_df_CPTAC.transpose()
    
    alive_C3 = ["C3L-00010","C3L-00026","C3L-00097","C3L-00359","C3L-00360","C3L-00418","C3L-00447","C3N-00312","C3N-00313"]
    dead_C3 = ["C3L-00011","C3L-00079","C3L-00096","C3L-00369","C3L-01302","C3N-00314","C3N-00491","C3N-00831","C3N-01200","C3L-00088","C3L-01287","C3N-01220"]
    FPKM_project_df_CPTAC = get_FPKM_values(metadata,"all_TCGA_projects\\",dead_C3,alive_C3)
    #genes = projects_res2[model][max_num]['genes']
    FPKM_project_df_CPTAC_renal_t = FPKM_project_df_CPTAC.transpose()
    #sorting the features by importance and running the model using reverse feature selection
    for fet_num in range(len(rna_top_fet.columns),0,-1):
        X,Y = get_X_Y(rna_top_fet.loc[alive_dead_ls_train,:].fillna(0),alive_ls,dead_ls)
        from sklearn.ensemble import RandomForestClassifier
        #clf_best_clf_1k = RandomForestClassifier(max_depth=2, random_state=0, n_estimators=400)
        clf_best_clf_1k.fit(X, Y)
        #test the model
        X,Y = get_X_Y(rna_top_fet.loc[alive_dead_ls_val,:].fillna(0),alive_ls,dead_ls)
        pred = clf_best_clf_1k.predict_proba(X)
        y = np.array(Y)
        pred = pred[:,1]
        auc = get_AUC(y,pred)
        #auc_4proj.append(auc)
        top_genes_list = list(rna_top_fet.columns)
        
        alive_C3 = ["C3L-00008","C3L-00032","C3L-00098","C3L-00136","C3L-00137","C3L-00143","C3L-00145","C3L-00156","C3L-00157","C3L-00362","C3L-00413","C3L-00449","C3L-00905","C3L-00921","C3L-00942","C3L-00946","C3L-00949","C3L-00961","C3L-01312","C3N-00323","C3N-00880"]
        dead_C3 = ["C3L-00771","C3L-01307","C3L-00139","C3N-00200","C3N-01349","C3N-01520","C3N-01003"]
        FPKM_project_df_CPTAC_uter_t = FPKM_project_df_CPTAC_uter_t[top_genes_list]
        #X,Y = get_X_Y(FPKM_project_df_t.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_C3)
        X,Y =get_X_Y(FPKM_project_df_CPTAC_uter_t.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_C3)
        pred = clf_best_clf_1k.predict_proba(X)
        #print("pred",pred,Y)
        y = np.array(Y)
        pred = pred[:,1]
        auc_c3_UCEC = get_AUC(y,pred)
        #if len(top_genes_list)%5==0:
        #    print("prject",TCGA_project,"top features",len(top_genes_list),"AUC-ROC",auc,"CPTAC_AUC",auc_c3)
        #project_models_and_scores[fet_num]={"auc_val_score":auc,"clf":clf_best_clf_1k, "genes":top_genes_list,"CPTAC_AUC":auc_c3}

        alive_C3 = ["C3L-00010","C3L-00026","C3L-00097","C3L-00359","C3L-00360","C3L-00418","C3L-00447","C3N-00312","C3N-00313"]
        dead_C3 = ["C3L-00011","C3L-00079","C3L-00096","C3L-00369","C3L-01302","C3N-00314","C3N-00491","C3N-00831","C3N-01200","C3L-00088","C3L-01287","C3N-01220"]
  
        FPKM_project_df_CPTAC_renal_t = FPKM_project_df_CPTAC_renal_t[top_genes_list]
        #X,Y = get_X_Y(FPKM_project_df_t.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_C3)
        X,Y =get_X_Y(FPKM_project_df_CPTAC_renal_t.loc[alive_C3+dead_C3,:].fillna(0),alive_C3,dead_C3)
        pred = clf_best_clf_1k.predict_proba(X)
        #print("pred",pred,Y)
        y = np.array(Y)
        pred = pred[:,1]
        auc_c3_renal = get_AUC(y,pred)
        #if len(top_genes_list)%5==0:
        #    print("prject",TCGA_project,"top features",len(top_genes_list),"AUC-ROC",auc,"CPTAC_AUC",auc_c3)
        #project_models_and_scores[fet_num]={"auc_val_score":auc,"clf":clf_best_clf_1k, "genes":top_genes_list,"CPTAC_AUC":auc_c3}

        if len(top_genes_list)%5==0:
            print("prject",TCGA_project,"top features",len(top_genes_list),"AUC-ROC",auc,"clf_n_fet",clf_best_clf_1k.n_features_,"CPTAC_AUC_renal",auc_c3_renal, "auc_c3_UCEC",auc_c3_UCEC)
        project_models_and_scores[fet_num]={"auc_val_score":auc,"clf":clf_best_clf_1k, "genes":top_genes_list,"CPTAC_AUC_renal":auc_c3_renal, "auc_c3_UCEC":auc_c3_UCEC}
        #print(top_genes_list)
        #print("-------------------------------------------------------------")

        #removing top features and calculating
        genes_importance = pd.DataFrame(clf_best_clf_1k.feature_importances_, index = rna_top_fet.columns )
        genes_importance_sorted = genes_importance.sort_values(by =[0],ascending=False)
        top_genes_list = genes_importance_sorted.index[:fet_num-1]
        rna_top_fet = rna_top_fet[top_genes_list]
    import pickle
    
    #projects_res[TCGA_project] = project_models_and_scores
    filename = 'dogs'
    outfile = open(TCGA_project+".pickle",'wb')
    pickle.dump(project_models_and_scores,outfile)
    outfile.close()

    

    # if the auc is more than 0.7 continue with the selection
    
    #clf,auc_val,auc_train_data,auc_c3,genes_importance_sorted_ls = RF_moudule_GridSearch(FPKM_project_df_t,dead_ls,alive_ls)
    if auc_val>0.7:
        genes_importance_sorted_fltr = genes_importance_sorted[genes_importance_sorted[0]>0]
        #if the auc is bigger than auc=0.7 we can refine the model to get better results
        RF_moudule_GridSearch(FPKM_project_df_t[list(genes_importance_sorted_fltr.index)],dead_ls,alive_ls)
        projects_auc_0_7.append(TCGA_project)
        genes_importance = pd.DataFrame(clf.feature_importances_, index = FPKM_project_df_t.columns)
        genes_importance_sorted = genes_importance.sort_values(by =[0],ascending=False)
    #print(genes_importance_sorted)

    #removing the dot from the index and removing all the features that got importance score of 0.0
    genes_importance_sorted_fltr = genes_importance_sorted[genes_importance_sorted[0]>0]
    genes_importance_sorted_fltr["index_c"] = genes_importance_sorted_fltr.index
    genes_importance_sorted_fltr = genes_importance_sorted_fltr.join(genes_importance_sorted_fltr["index_c"].str.split(".").apply(pd.Series),rsuffix="real_index")
    genes_importance_sorted_fltr = genes_importance_sorted_fltr.set_index("0real_index")
    genes_importance_sorted_fltr = genes_importance_sorted_fltr.drop(["index_c",1], axis=1)
    #assiging the Gene name to for each ENSG ID
    genes_importance_sorted_fltr_ENSG = genes_importance_sorted_fltr.join(ENSG_key)
    #break"""

    #print(FPKM_project_df.head(3))
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-7-fa3faa9e71ec>, line 178)